In [1]:
#Import Modules
from logger import setup_custom_logger
import pandas as pd
import requests
import cbsodata
import datetime

In [2]:
# Setup Logger
try:
    logger = setup_custom_logger("E002_Unemployment_Rate")
    logger.info('starting')
except:
    logger.exception('logger could not be loaded')
    raise

2021-03-23 13:50:33 INFO     E002_Unemployment_Rate starting


INFO:E002_Unemployment_Rate:starting


In [3]:
try:
    # Get current date information
    now = datetime.datetime.now()
    
    logger.info('datetime loaded')
    yearMin = now.year-3
    yearMax = now.year
    
    # Values you can load
    # now.year
    # now.month
    # now.day
    # now.hour
    # now.minute
except:
    logger.exception('datetime could not be loaded')
    logger.info('set yearmin to a default value')
    # Set default values for fallback
    yearMin = 2010
    yearMax = 2030
    raise



2021-03-23 13:50:33 INFO     E002_Unemployment_Rate datetime loaded


INFO:E002_Unemployment_Rate:datetime loaded


In [4]:
# Dataset 80590ned
dataset_id = '80590ned'

# Table definitions

# Geslacht: T001038 = Totaal (man/vrouw)
# Leeftijd: 52052   = 15 tot 75 jaar
# Perioden: YYYY%%MM
# Seizoengecorrigeerd_2 = x1000_Beroepsbevolking_Seizoengecorrigeerd
# Seizoengecorrigeerd_4 = x1000_Werkzame_Beroepsbevolking_Seizoengecorrigeerd
# Seizoengecorrigeerd_6 = x1000_Werkloze_Beroepsbevolking_Seizoengecorrigeerd
# Seizoengecorrigeerd_8 = Werkloosheidspeercentage_Seizoengecorrigeerd_procenten
# Seizoengecorrigeerd_12 = Bruto_Arbeitsparticipatie_Seizoengecorrigeerd_procenten
# Seizoengecorrigeerd_14 = Netto_Arbeitsparticipatie_Seizoengecorrigeerd_procenten


In [5]:
try:
    logger.info(f'Retrieve data from dataset {dataset_id}')
    data = pd.DataFrame(cbsodata.get_data(
    f'{dataset_id}',
    filters=f"Geslacht eq 'T001038' and Leeftijd eq '52052   ' and substring(Perioden,0,4) ge '{yearMin}'", # These spaces need to be there, they are part of the value..
    select=["Geslacht", "Leeftijd", "Perioden", "Seizoengecorrigeerd_2", "Seizoengecorrigeerd_4", "Seizoengecorrigeerd_6", "Seizoengecorrigeerd_8", "Seizoengecorrigeerd_12", "Seizoengecorrigeerd_14" ]))
except:
    logger.exception('error loading data from CBS Statline')
    raise

2021-03-23 13:50:33 INFO     E002_Unemployment_Rate Retrieve data from dataset 80590ned


INFO:E002_Unemployment_Rate:Retrieve data from dataset 80590ned


In [6]:
# Remove quaterly and yearly data
try:
    data = data[data["Perioden"].str.contains("kwartaal")==False]
    data = data[data['Perioden'].map(len) > 4 ]
except:
    logger.exception('Perioden filter could not be applied')
    raise

In [7]:
# Rename columns
try:
    data["Seizoengecorrigeerd_2"] = data["Seizoengecorrigeerd_2"] * 1000
    data["Seizoengecorrigeerd_4"] = data["Seizoengecorrigeerd_4"] * 1000
    data["Seizoengecorrigeerd_6"] = data["Seizoengecorrigeerd_6"] * 1000
except:
    logger.exception('Columns could not be * 1000')
    raise

In [8]:
try:
    data = data.rename(columns={
    "Seizoengecorrigeerd_2": "Beroepsbevolking_Seizoengecorrigeerd",
    "Seizoengecorrigeerd_4" : "Werkzame_Beroepsbevolking_Seizoengecorrigeerd",
    "Seizoengecorrigeerd_6" : "Werkloze_Beroepsbevolking_Seizoengecorrigeerd",
    "Seizoengecorrigeerd_8" : "Werkloosheidspeercentage_Seizoengecorrigeerd_procenten",
    "Seizoengecorrigeerd_12" : "Bruto_Arbeitsparticipatie_Seizoengecorrigeerd_procenten", 
    "Seizoengecorrigeerd_14" : "Netto_Arbeitsparticipatie_Seizoengecorrigeerd_procenten"
})
except:
    logger.exception('Columns could not be renamed')
    raise

In [9]:
# Export dataFrame to Excel file
try:
    data.to_csv("output/E002-UnemploymentRate.csv")  
except:
    logger.exception('dataFrame could not be exported to output folder')